GWAS summery stat data preprocess
1.need to convert Markername to rsID by combining Chr and basepair coordinate in _lifted bim file and and replace Markername by matching
2.calculate z-score using P-value and effect size using munge_sumstat in ldsc github repository

In [1]:
#read meta5 file -- this is our GWAS summery stat data
! zless /data/CARD/PD/summary_stats/META5_all.gz

MarkerName      Allele1 Allele2 Freq1   FreqSE  MinFreq MaxFreq Effect  StdErr  P-value Direction       HetISq  HetChiSq        HetDf   HetPVal freqSpan
chr11:88249377  t       c       0.9905  0.0026  0.9882  0.9971  0.0238  0.0594  0.6889  -+?+++--+???-+++?       20      13.753  11      0.247   0.00890000000000002
chr1:60320992   a       g       0.9375  0.0049  0.9221  0.9473  0.0185  0.021   0.3801  +-?--+?++??-+---+       20.1    15.022  12      0.2402  0.0252
chr8:135908647  a       g       0.2112  0.007   0.1703  0.2316  -0.0054 0.0123  0.6598  --?+++?--??-----+       27.2    16.492  12      0.1697  0.0613
chr16:77148858  a       g       0.9963  5e-04   0.9961  0.9984  -0.0228 0.1356  0.8663  -??-+??+-???----?       0       4.331   8       0.8261  0.00229999999999997
chr11:97895884  c       g       0.0587  0.0035  0.0361  0.0685  -0.0219 0.0217  0.3148  +-?--+?--??+---++       0.2     12.029  12      0.4433  0.0324
chr2:80117945   a       g       0.9979  7e-04   0.9968  0.9988  -0

In [ ]:
#copy meta 5 file into data folder - run only once for download
! cp -R /data/CARD/PD/summary_stats/META5_all.gz /data/songy4/TWAS/data/

#unzip META5_all.gz - run only once for download
! gzip -d META5_all.gz

#make sumstat.csv which contains MarkerName, Allele1, Allele2, Effect, StdErr, P-value
! awk '{print $1, $2, $3, $8, $9, $10}' data_folder/META5_all > data_folder/sumstat.txt

In [2]:
! awk '{print $1, $2, $3, $8, $9, $10}' data_folder/META5_all > data_folder/sumstat.txt

^C


In [2]:
#import library
import subprocess
import sys
import os
import shutil
import pandas as pd
import numpy as np

# set paths
basedir = '/data/songy4/proteomics_196'
datadir = f'{basedir}/data_folder'
twasdir = f'/data/songy4/twas'
fusiondir = f'{twasdir}/fusion_twas'
geno_path = f'{datadir}/qc_proteomics196_csf_cardio'
###Add _case_hg19_lifted or _control_hg19_lifted
gene_list_path = f'{datadir}/PDBP_PPMI_proteomics_csf_cardio_gene_list.txt'
pheno_path = f'{datadir}/PDBP_PPMI_proteomics_196_csf_cardio_npx_expression'
###Add _case.txt or _control.txt
coord_path = f'{datadir}/PDBP_PPMI_proteomics_csf_cardio_coordinate.txt'
covar_path = f'{datadir}/covariates_csf_cardio'

Read meta data

In [3]:
statsum = pd.read_csv(f"{twasdir}/data_folder/META5_all", sep='\t')
print('shape of META5_all:', statsum.shape)
statsum.head()

shape of META5_all: (11477547, 16)


,MarkerName,Allele1,Allele2,Freq1,FreqSE,MinFreq,MaxFreq,Effect,StdErr,P-value,Direction,HetISq,HetChiSq,HetDf,HetPVal,freqSpan
0,chr11:88249377,t,c,0.9905,0.0026,0.9882,0.9971,0.0238,0.0594,0.6889,-+?+++--+???-+++?,20.0,13.753,11,0.2470,0.0089
1,chr1:60320992,a,g,0.9375,0.0049,0.9221,0.9473,0.0185,0.0210,0.3801,+-?--+?++??-+---+,20.1,15.022,12,0.2402,0.0252
2,chr8:135908647,a,g,0.2112,0.0070,0.1703,0.2316,-0.0054,0.0123,0.6598,--?+++?--??-----+,27.2,16.492,12,0.1697,0.0613
3,chr16:77148858,a,g,0.9963,0.0005,0.9961,0.9984,-0.0228,0.1356,0.8663,-??-+??+-???----?,0.0,4.331,8,0.8261,0.0023
4,chr11:97895884,c,g,0.0587,0.0035,0.0361,0.0685,-0.0219,0.0217,0.3148,+-?--+?--??+---++,0.2,12.029,12,0.4433,0.0324


In [7]:
#count if Freq1 is bigger than 0.5 --> this is to check if minor allele is A1
!cat {twasdir}/data_folder/META5_all | awk -F"\t" '$4>0.5' | wc -l

#check again how many Freq1 
statsum_flip = statsum[statsum.Freq1 > 0.5]
print(statsum_flip.head())

4973527
       MarkerName Allele1 Allele2   Freq1  FreqSE  MinFreq  MaxFreq  Effect  \
0  chr11:88249377       t       c  0.9905  0.0026   0.9882   0.9971  0.0238   
1   chr1:60320992       a       g  0.9375  0.0049   0.9221   0.9473  0.0185   
3  chr16:77148858       a       g  0.9963  0.0005   0.9961   0.9984 -0.0228   
5   chr2:80117945       a       g  0.9979  0.0007   0.9968   0.9988 -0.4366   
6    chr18:814730       c       g  0.8893  0.0075   0.8639   0.9086 -0.0022   

   StdErr  P-value          Direction  HetISq  HetChiSq  HetDf  HetPVal  \
0  0.0594   0.6889  -+?+++--+???-+++?    20.0    13.753     11   0.2470   
1  0.0210   0.3801  +-?--+?++??-+---+    20.1    15.022     12   0.2402   
3  0.1356   0.8663  -??-+??+-???----?     0.0     4.331      8   0.8261   
5  0.4506   0.3326  ?????+?--???---+?     0.0     5.236      6   0.5139   
6  0.0176   0.9025  --?---?+-??+++--+     0.0     4.652     12   0.9686   

   freqSpan  
0    0.0089  
1    0.0252  
3    0.0023  
5    0.002

In [9]:
#flip Allele1 and Allele2 if Freq1 is bigger than .5 and change Freq1 to negative
stat_sum = statsum.copy()
cond = (stat_sum['Freq1'] > 0.5)
stat_sum.loc[cond, ['Allele1', 'Allele2']] = stat_sum.loc[cond, ['Allele2', 'Allele1']].values
#stat_sum_flip.Allele1, stat_sum_flip.Allele2 = np.where(stat_sum_flip.Freq1 > 0.5, [stat_sum_flip.Allele2, stat_sum_flip.Allele1], [stat_sum_flip.Allele1, stat_sum_flip.Allele2])

#change Freq1
stat_sum.loc[cond, ['Freq1']] = 1-stat_sum['Freq1']

#change Effect value
stat_sum.loc[cond, 'Effect'] *= -1

#check again how many Freq1 are smaller than 0.01
statsum_useless = stat_sum[stat_sum.Freq1 < 0.01]
print(statsum_useless.shape)

#drop two rows Freq1 is smaller than 0.01
stat_sum.drop(stat_sum[stat_sum.Freq1 < 0.01].index, inplace=True)
stat_sum = stat_sum.reset_index(drop=True)
#stat_sum = stat_sum.drop(['index'], axis=1)

#drop unwanted columns
stat_sum.drop(stat_sum.iloc[:,10:16], axis=1, inplace=True)
stat_sum.drop(stat_sum.iloc[:,4:7], axis=1, inplace=True)

#check the unique number of MarkerName in meta
print("Unique MarkerName in meta data:", stat_sum['MarkerName'].nunique())

stat_sum
##no need to change  StdErr, P-value

(3693134, 16)
Unique MarkerName in meta data: 7784413


,MarkerName,Allele1,Allele2,Freq1,Effect,StdErr,P-value
0,chr1:60320992,g,a,0.0625,-0.0185,0.0210,0.380100
1,chr8:135908647,a,g,0.2112,-0.0054,0.0123,0.659800
2,chr11:97895884,c,g,0.0587,-0.0219,0.0217,0.314800
3,chr18:814730,g,c,0.1107,0.0022,0.0176,0.902500
4,chr10:120407145,c,t,0.0888,0.0278,0.0179,0.120300
...,...,...,...,...,...,...,...
7784408,chr18:50482478,g,c,0.0278,0.0203,0.0319,0.525300
7784409,chr11:47906648,c,t,0.0548,0.0062,0.0207,0.765700
7784410,chr3:47855594,c,t,0.3501,-0.0348,0.0105,0.000944
7784411,chr3:194372962,a,g,0.0166,0.0268,0.0500,0.591900


In [10]:
#save stat_sum 
stat_sum.to_csv(r'./data_folder/sumstat.txt', sep='\t' ,index=False)

In [11]:
def shell_do(command, log=False, return_log=False):
    print(f'Executing: {(" ").join(command.split())}', file=sys.stderr)

    res=subprocess.run(command.split(), stdout=subprocess.PIPE)

    if log:
        print(res.stdout.decode('utf-8'))
    if return_log:
        return(res.stdout.decode('utf-8'))

Read Bim file

In [12]:
#open bim file 
bim_df = pd.read_csv(f"{geno_path}_case_hg19_lifted.bim", sep='\t', header=None,  names=('chr', 'rsid', 'kb', 'pos', 'a1', 'a2'))

#combine chr and pos columns
bim_df['MarkerName'] = str('chr') + bim_df.chr.astype(str) + ':' + bim_df.pos.astype(str)

#check the shape of meta
print("shape of bim data:", bim_df.shape)

#check the shape of meta
print("number of unique MarkerName in bim data:", bim_df['MarkerName'].nunique())
print("number of unique rsid in bim data:", bim_df['rsid'].nunique())
bim_df.head()

shape of bim data: (23060234, 7)
number of unique MarkerName in bim data: 23060188
number of unique rsid in bim data: 23060234


,chr,rsid,kb,pos,a1,a2,MarkerName
0,1,rs145427775,0,10291,T,C,chr1:10291
1,1,rs55998931,0,10492,T,C,chr1:10492
2,1,rs199896944,0,13504,A,G,chr1:13504
3,1,rs199856693,0,14933,A,G,chr1:14933
4,1,rs201855936,0,14948,A,G,chr1:14948


In [13]:
#check duplicated rows
bim_df_dup = bim_df[bim_df['MarkerName'].duplicated(keep=False)]
#check unique marker name and a1, a2 in duplicated rows
print('Number of unique MarkerName in duplicated bim_df: ', bim_df_dup['MarkerName'].nunique())
print('Unique MarkerName in duplicated bim_df: ', bim_df_dup['MarkerName'].unique())
print('Unique allele 1 in duplicated bim_df: ', bim_df_dup['a1'].unique())
print('Unique allele 2 in duplicated bim_df: ', bim_df_dup['a2'].unique())
## there are 46 duplicated MarkerName with 2 different rsid

Number of unique MarkerName in duplicated bim_df:  46
Unique MarkerName in duplicated bim_df:  ['chr1:148609803' 'chr1:148561459' 'chr3:195218551' 'chr3:195216671'
 'chr3:195216763' 'chr3:195218266' 'chr3:195216747' 'chr3:195229571'
 'chr3:195229612' 'chr9:70948646' 'chr9:70933437' 'chr9:44409306'
 'chr9:44093596' 'chr9:44065499' 'chr9:44060266' 'chr10:46914775'
 'chr10:46911590' 'chr10:46906700' 'chr11:1093136' 'chr14:19819097'
 'chr14:22629698' 'chr14:106478074' 'chr14:106478531' 'chr14:106478574'
 'chr14:106478627' 'chr15:21164636' 'chr22:19172321' 'chr22:19198488'
 'chr22:19257315' 'chr22:19263547' 'chr22:19299612' 'chr22:19300219'
 'chr22:19310505' 'chr22:19390804' 'chr22:19497230' 'chr22:19503090'
 'chr22:19555053' 'chr22:19593171' 'chr22:19623380' 'chr22:19643104'
 'chr22:19669438' 'chr22:19719099' 'chr22:19738319' 'chr22:19739438'
 'chr22:19786130' 'chr22:18872544']
Unique allele 1 in duplicated bim_df:  ['A' 'T' 'C' '*' 'G' 'AAGAC']
Unique allele 2 in duplicated bim_df:  ['G' 

Merge meta data and bim file to create required format of GWAS sumstat 

In [14]:
#merge stat_sum with bim_df
meta_bim = pd.merge(stat_sum, bim_df, on=['MarkerName'], how='inner')
meta_bim

,MarkerName,Allele1,Allele2,Freq1,Effect,StdErr,P-value,chr,rsid,kb,pos,a1,a2
0,chr1:60320992,g,a,0.0625,-0.0185,0.0210,0.380100,1,rs116406626,0,60320992,G,A
1,chr8:135908647,a,g,0.2112,-0.0054,0.0123,0.659800,8,rs11992603,0,135908647,A,G
2,chr10:120407145,c,t,0.0888,0.0278,0.0179,0.120300,10,rs4336949,0,120407145,C,T
3,chr11:29372878,t,c,0.0592,0.0049,0.0218,0.822100,11,rs72638174,0,29372878,T,C
4,chr3:164053059,a,g,0.0146,0.0607,0.0488,0.214100,3,rs115079612,0,164053059,A,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6478463,chr16:84525085,t,c,0.0268,-0.0387,0.0310,0.211700,16,rs2927754,0,84525085,T,C
6478464,chr11:47906648,c,t,0.0548,0.0062,0.0207,0.765700,11,rs148423893,0,47906648,C,T
6478465,chr3:47855594,c,t,0.3501,-0.0348,0.0105,0.000944,3,rs62263573,0,47855594,C,T
6478466,chr3:194372962,a,g,0.0166,0.0268,0.0500,0.591900,3,rs149847890,0,194372962,A,G


In [15]:
#save stat_sum 
meta_bim.to_csv(r'./data_folder/meta_bim.txt', sep='\t' ,index=False)

In [17]:
#check duplicated rsid
meta_bim_dup = meta_bim[meta_bim['rsid'].duplicated(keep=False)]
print('number of unique rsid in duplicated meta_bim: ', meta_bim_dup['rsid'].nunique())
#check number of unique rsid just in case
print('number of unique rsid in meta_bim: ', meta_bim.rsid.nunique())
#check number of unique MarkerName just in case
print('number of unique MarkerName in meta_bim: ', meta_bim.MarkerName.nunique())

number of unique rsid in duplicated meta_bim:  0
number of unique rsid in meta_bim:  6478468
number of unique MarkerName in meta_bim:  6478463


In [19]:
#print gwas summery stat with SNP that has duplicated MarkerName
dup_marker = bim_df_dup['MarkerName'].unique().tolist()
snp_stat_dup = meta_bim[meta_bim['MarkerName'].isin(dup_marker)]
snp_stat_dup
#5 duplicated (MarkerName) position with different rsid

,MarkerName,Allele1,Allele2,Freq1,Effect,StdErr,P-value,chr,rsid,kb,pos,a1,a2
3011341,chr22:19503090,c,t,0.0111,-0.0449,0.0637,0.48050,22,rs370330113,0,19503090,C,T
3011342,chr22:19503090,c,t,0.0111,-0.0449,0.0637,0.48050,22,rs138605059,0,19503090,C,T
3214210,chr22:19172321,t,g,0.0738,0.0117,0.0183,0.52360,22,rs199687134,0,19172321,C,T
3214211,chr22:19172321,t,g,0.0738,0.0117,0.0183,0.52360,22,rs1780641,0,19172321,T,G
3279286,chr22:19497230,c,t,0.0475,-0.0466,0.0263,0.07624,22,rs2521701,0,19497230,T,C
3279287,chr22:19497230,c,t,0.0475,-0.0466,0.0263,0.07624,22,rs117551893,0,19497230,C,T
3950528,chr22:19390804,c,t,0.0914,-0.0161,0.0164,0.32640,22,rs141585725,0,19390804,T,C
3950529,chr22:19390804,c,t,0.0914,-0.0161,0.0164,0.32640,22,rs11089274,0,19390804,C,T
5019123,chr22:19643104,c,t,0.1720,-0.0006,0.0134,0.96150,22,rs145711952,0,19643104,A,G
5019124,chr22:19643104,c,t,0.1720,-0.0006,0.0134,0.96150,22,rs9618649,0,19643104,C,T


In [20]:
#drop rows in meta if rsid is NaN 
#meta_df = snp_stat[snp_stat['SNP'].notna()]

sumstat_df = meta_bim.copy()
#change Allele1 and Allele2 to A1 and A2
sumstat_df = sumstat_df.rename(columns={'Allele1':'A1', 'Allele2':'A2','rsid':'SNP'})
sumstat_df = sumstat_df.drop(['a1', 'a2', 'kb'], axis=1)

#reset index
#meta_df = meta_df.reset_index(drop=True)

#check numbers of rows in meta_df
print("numbers of rows in meta data:", sumstat_df.shape[0])

sumstat_df.head()

numbers of rows in meta data: 6478468


,MarkerName,A1,A2,Freq1,Effect,StdErr,P-value,chr,SNP,pos
0,chr1:60320992,g,a,0.0625,-0.0185,0.0210,0.3801,1,rs116406626,60320992
1,chr8:135908647,a,g,0.2112,-0.0054,0.0123,0.6598,8,rs11992603,135908647
2,chr10:120407145,c,t,0.0888,0.0278,0.0179,0.1203,10,rs4336949,120407145
3,chr11:29372878,t,c,0.0592,0.0049,0.0218,0.8221,11,rs72638174,29372878
4,chr3:164053059,a,g,0.0146,0.0607,0.0488,0.2141,3,rs115079612,164053059


In [21]:
#calculate z score with effect size and standard error (z = (score-mean)/se)

def z_score(a, b):
    if b == 0:
        return 0.0
    return float(a)/b

sumstat_df['Z'] = np.vectorize(z_score)(sumstat_df['Effect'], sumstat_df['StdErr'])

sumstat_df

,MarkerName,A1,A2,Freq1,Effect,StdErr,P-value,chr,SNP,pos,Z
0,chr1:60320992,g,a,0.0625,-0.0185,0.0210,0.380100,1,rs116406626,60320992,-0.880952
1,chr8:135908647,a,g,0.2112,-0.0054,0.0123,0.659800,8,rs11992603,135908647,-0.439024
2,chr10:120407145,c,t,0.0888,0.0278,0.0179,0.120300,10,rs4336949,120407145,1.553073
3,chr11:29372878,t,c,0.0592,0.0049,0.0218,0.822100,11,rs72638174,29372878,0.224771
4,chr3:164053059,a,g,0.0146,0.0607,0.0488,0.214100,3,rs115079612,164053059,1.243852
...,...,...,...,...,...,...,...,...,...,...,...
6478463,chr16:84525085,t,c,0.0268,-0.0387,0.0310,0.211700,16,rs2927754,84525085,-1.248387
6478464,chr11:47906648,c,t,0.0548,0.0062,0.0207,0.765700,11,rs148423893,47906648,0.299517
6478465,chr3:47855594,c,t,0.3501,-0.0348,0.0105,0.000944,3,rs62263573,47855594,-3.314286
6478466,chr3:194372962,a,g,0.0166,0.0268,0.0500,0.591900,3,rs149847890,194372962,0.536000


In [22]:
#function to move columns
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

#rearrange Gene_Symbol, Chr, Start, Stop columns right nextg to index column 
sumstat = movecol(sumstat_df, cols_to_move=['SNP'], ref_col='MarkerName', place='After')

#drop MarkerName, Effect, StdErr, P-value colunms 
sumstat = sumstat.drop(['MarkerName', 'Effect', 'StdErr', 'P-value', 'chr', 'pos'], axis=1)

sumstat = sumstat.reset_index(drop=True)
print("shape of meta data:", sumstat.shape)
sumstat.head()

## this sumstat is the format we will use for our fusion analysis ###

shape of meta data: (6478468, 5)


,SNP,A1,A2,Freq1,Z
0,rs116406626,g,a,0.0625,-0.880952
1,rs11992603,a,g,0.2112,-0.439024
2,rs4336949,c,t,0.0888,1.553073
3,rs72638174,t,c,0.0592,0.224771
4,rs115079612,a,g,0.0146,1.243852


In [24]:
#calculate chisq
stat_sum_chi = sumstat_df.copy()
stat_sum_chi['chisq'] = stat_sum_chi['Z']**2
stat_sum_chi

,MarkerName,A1,A2,Freq1,Effect,StdErr,P-value,chr,SNP,pos,Z,chisq
0,chr1:60320992,g,a,0.0625,-0.0185,0.0210,0.380100,1,rs116406626,60320992,-0.880952,0.776077
1,chr8:135908647,a,g,0.2112,-0.0054,0.0123,0.659800,8,rs11992603,135908647,-0.439024,0.192742
2,chr10:120407145,c,t,0.0888,0.0278,0.0179,0.120300,10,rs4336949,120407145,1.553073,2.412035
3,chr11:29372878,t,c,0.0592,0.0049,0.0218,0.822100,11,rs72638174,29372878,0.224771,0.050522
4,chr3:164053059,a,g,0.0146,0.0607,0.0488,0.214100,3,rs115079612,164053059,1.243852,1.547169
...,...,...,...,...,...,...,...,...,...,...,...,...
6478463,chr16:84525085,t,c,0.0268,-0.0387,0.0310,0.211700,16,rs2927754,84525085,-1.248387,1.558470
6478464,chr11:47906648,c,t,0.0548,0.0062,0.0207,0.765700,11,rs148423893,47906648,0.299517,0.089710
6478465,chr3:47855594,c,t,0.3501,-0.0348,0.0105,0.000944,3,rs62263573,47855594,-3.314286,10.984490
6478466,chr3:194372962,a,g,0.0166,0.0268,0.0500,0.591900,3,rs149847890,194372962,0.536000,0.287296


In [28]:
#calculate lambda (median(z^2) / 0.455) using chisq
lambda_stat_sum = ((stat_sum_chi['chisq'].median())/0.455)
print("lambda from chisq:",lambda_stat_sum)

#calculate lambda (median(z^2) / 0.455) using pvalue
lambda_statsum = ((stat_sum_chi['P-value'].median())/0.456)
print("lambda from p value:",lambda_statsum)

lambda from chisq: 1.171208791208791
lambda from p value: 1.0206140350877193


In [48]:
#save meta_df as txt -- run only once for download
sumstat.to_csv(r'./data_folder/meta.txt', sep='\t' ,index=False)
#gwas sumary stat is ready to use

Grab significant p value from sumstat 
## from here, run after 4_postprocess notebook

In [22]:
##grab significant GWAS hits (p <= 5e-8)
sig_meta = sumstat_df[sumstat_df['P-value']< 5e-8]
#reset index
sig_meta = sig_meta.reset_index(drop=True)
print("shape of significant GWAS data:", sig_meta.shape)

sig_meta.head()

shape of significant GWAS data: (5891, 10)


,MarkerName,A1,A2,Effect,StdErr,P-value,chr,SNP,pos,Z
0,chr16:31054607,t,c,-0.0831,0.0099,6.178000e-17,16,rs1108431,31054607,-8.393939
1,chr10:121534732,t,c,0.0587,0.0095,5.506000e-10,10,rs196202,121534732,6.178947
2,chr5:102446462,a,c,-0.0603,0.0103,4.716000e-09,5,rs257320,102446462,-5.854369
3,chr16:31023273,a,g,-0.0852,0.0100,1.338000e-17,16,rs12447930,31023273,-8.520000
4,chr4:951040,g,t,-0.1074,0.0097,1.054000e-28,4,rs3733344,951040,-11.072165


In [24]:
#check if double marker are in sig_meta
sig_meta_dup = sig_meta[sig_meta['MarkerName'].isin(double_marker)]
sig_meta_dup

,MarkerName,A1,A2,Effect,StdErr,P-value,SNP,Z


In [23]:
#save sig_meta -- run only once
sig_meta.to_csv(r'./data_folder/gwas_pos.csv', index=False)

In [29]:
#bring in coordinate data
coords = pd.read_csv(coord_path, sep='\t')
coords

,X.Chr,start,end,ID
0,1,1682671,1711896,NADK
1,1,11917521,11918988,NTproBNP
2,1,16268364,16302627,ZBTB17
3,1,20301925,20306932,PLA2G2A
4,1,22148738,22263790,HSPG2
...,...,...,...,...
338,21,33031935,33041244,SOD1
339,21,43731777,43735761,TFF3
340,21,45192393,45196326,CSTB
341,21,46305868,46351904,ITGB2


Read the 90 parkinson's GWAS hits from Mike's paper (some are in the supplement)
##https://www.thelancet.com/journals/laneur/article/PIIS1474-4422(19)30320-5/fulltext

In [30]:
#read 90 parkinson's GWAS hits
pd_hits = pd.read_csv(r"./data_folder/PD_GRS_dbSNPs.tab", sep='\t', lineterminator='\r', header=None,  names=('rsid', 'author', 'condition', 'name') )
pd_hits.drop(pd_hits.iloc[:,1:4], axis=1, inplace=True)
pd_hits['rsid'] = pd_hits['rsid'].str.replace('\n', '')
pd_hits['rsid'].replace('', np.nan, inplace=True)
pd_hits.dropna(subset=['rsid'], inplace=True)
pd_hits

,rsid
0,rs114138760
1,rs35749011
2,rs76763715
3,rs6658353
4,rs11578699
...,...
85,rs12456492
86,rs8087969
87,rs55818311
88,rs77351827


In [31]:
#merge rsid with pos from meta data
pd_hits_pos_1 = pd.merge(bim_df, pd_hits, on=['rsid'], how='inner')
print(pd_hits_pos_1.head())

#get the rsid without pos
mis_rsid = pd_hits[~pd_hits['rsid'].isin(pd_hits_pos_1['rsid'])]
mis_rsid = mis_rsid.reset_index(drop=True)
print("number of missing rsid: ", mis_rsid.rsid.nunique())
print("missing rsid: ", mis_rsid.rsid.tolist())

   chr        rsid  kb        pos a1 a2      MarkerName
0    1  rs35749011   0  155135036  A  G  chr1:155135036
1    1  rs76763715   0  155205634  C  T  chr1:155205634
2    1  rs11578699   0  171719769  T  C  chr1:171719769
3    1    rs823118   0  205723572  C  T  chr1:205723572
4    1  rs11557080   0  205737739  A  G  chr1:205737739
number of missing rsid:  14
missing rsid:  ['rs114138760', 'rs6658353', 'rs76116224', 'rs2042477', 'rs11683001', 'rs11707416', 'rs34025766', 'rs6825004', 'rs26431', 'rs76949143', 'rs6476434', 'rs6497339', 'rs2904880', 'rs666463']


In [29]:
#merge rsid with pos from meta data

pd_hits_pos_2 = pd_hits.merge(bim_df, on=['rsid'], how='left') 
pd_hits_pos_2 = pd_hits_pos_2.drop(['kb'], axis=1)
pd_hits_pos_2

,rsid,chr,pos,a1,a2,MarkerName
0,rs114138760,NaN,NaN,NaN,NaN,NaN
1,rs35749011,1.0,155135036.0,A,G,chr1:155135036
2,rs76763715,1.0,155205634.0,C,T,chr1:155205634
3,rs6658353,NaN,NaN,NaN,NaN,NaN
4,rs11578699,1.0,171719769.0,T,C,chr1:171719769
...,...,...,...,...,...,...
85,rs12456492,18.0,40673380.0,G,A,chr18:40673380
86,rs8087969,18.0,48683589.0,G,T,chr18:48683589
87,rs55818311,19.0,2341047.0,C,T,chr19:2341047
88,rs77351827,20.0,6006041.0,T,C,chr20:6006041


In [30]:
##Fill in missing information manually by searching dbSNP NCBI
#input data for rs114138760
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs114138760', 'chr'] = '1'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs114138760', 'pos'] = '154898185'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs114138760', 'a1'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs114138760', 'a2'] = 'G'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs114138760', 'MarkerName'] = 'chr1:154898185'

#input data for rs6658353
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6658353', 'chr'] = '1'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6658353', 'pos'] = '161469054'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6658353', 'a1'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6658353', 'a2'] = 'G'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6658353', 'MarkerName'] = 'chr1:161469054'

#input data for rs76116224
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76116224', 'chr'] = '2'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76116224', 'pos'] = '18147848'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76116224', 'a1'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76116224', 'a2'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76116224', 'MarkerName'] = 'chr2:18147848'

#input data for rs2042477
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2042477', 'chr'] = '2'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2042477', 'pos'] = '96000943'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2042477', 'a1'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2042477', 'a2'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2042477', 'MarkerName'] = 'chr2:96000943'

#input data for rs11683001
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11683001', 'chr'] = '2'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11683001', 'pos'] = '102396963'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11683001', 'a1'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11683001', 'a2'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11683001', 'MarkerName'] = 'chr2:102396963'

#input data for rs11707416
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11707416', 'chr'] = '3'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11707416', 'pos'] = '151108965'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11707416', 'a1'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11707416', 'a2'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs11707416', 'MarkerName'] = 'chr3:151108965'

#input data for rs34025766
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs34025766', 'chr'] = '4'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs34025766', 'pos'] = '17968811'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs34025766', 'a1'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs34025766', 'a2'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs34025766', 'MarkerName'] = 'chr4:17968811'

#input data for rs6825004
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6825004', 'chr'] = '4'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6825004', 'pos'] = '77110365'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6825004', 'a1'] = 'G'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6825004', 'a2'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6825004', 'MarkerName'] = 'chr4:77110365'

#input data for rs26431
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs26431', 'chr'] = '5'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs26431', 'pos'] = '102365794'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs26431', 'a1'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs26431', 'a2'] = 'G'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs26431', 'MarkerName'] = 'chr5:102365794'

#input data for rs76949143
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76949143', 'chr'] = '7'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76949143', 'pos'] = '66009851'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76949143', 'a1'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76949143', 'a2'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs76949143', 'MarkerName'] = 'chr7:66009851'

#input data for rs6476434
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6476434', 'chr'] = '9'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6476434', 'pos'] = '34046391'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6476434', 'a1'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6476434', 'a2'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6476434', 'MarkerName'] = 'chr9:34046391'

#input data for rs6497339
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6497339', 'chr'] = '16'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6497339', 'pos'] = '19277493'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6497339', 'a1'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6497339', 'a2'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs6497339', 'MarkerName'] = 'chr16:19277493'

#input data for rs2904880
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2904880', 'chr'] = '16'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2904880', 'pos'] = '28944396'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2904880', 'a1'] = 'G'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2904880', 'a2'] = 'C'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs2904880', 'MarkerName'] = 'chr16:28944396'

#input data for rs666463
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs666463', 'chr'] = '17'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs666463', 'pos'] = '76425480'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs666463', 'a1'] = 'T'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs666463', 'a2'] = 'A'
pd_hits_pos_2.loc[pd_hits_pos_2.rsid == 'rs666463', 'MarkerName'] = 'chr17:76425480'

#convert column values to int
col_case=pd_hits_pos_2.columns[1:3]
pd_hits_pos_2.loc[:,col_case]=pd_hits_pos_2.loc[:,col_case].astype(np.int64)

pd_hits_pos_2

,rsid,chr,pos,a1,a2,MarkerName
0,rs114138760,1,154898185,C,G,chr1:154898185
1,rs35749011,1,155135036,A,G,chr1:155135036
2,rs76763715,1,155205634,C,T,chr1:155205634
3,rs6658353,1,161469054,C,G,chr1:161469054
4,rs11578699,1,171719769,T,C,chr1:171719769
...,...,...,...,...,...,...
85,rs12456492,18,40673380,G,A,chr18:40673380
86,rs8087969,18,48683589,G,T,chr18:48683589
87,rs55818311,19,2341047,C,T,chr19:2341047
88,rs77351827,20,6006041,T,C,chr20:6006041


In [81]:
#save pd_hits_pos_2 
pd_hits_pos_2.to_csv(r'./data_folder/PD_GRS_dbSNPs_position.csv', index=False)

In [31]:
#create pos_less and pos_more which are position plus minus 1 megabase
pd_hits_pos_2['pos_less']= pd_hits_pos_2['pos'] - 1000000
pd_hits_pos_2['pos_more']= pd_hits_pos_2['pos'] + 1000000
pd_hits_pos_2

,rsid,chr,pos,a1,a2,MarkerName,pos_less,pos_more
0,rs114138760,1,154898185,C,G,chr1:154898185,153898185,155898185
1,rs35749011,1,155135036,A,G,chr1:155135036,154135036,156135036
2,rs76763715,1,155205634,C,T,chr1:155205634,154205634,156205634
3,rs6658353,1,161469054,C,G,chr1:161469054,160469054,162469054
4,rs11578699,1,171719769,T,C,chr1:171719769,170719769,172719769
...,...,...,...,...,...,...,...,...
85,rs12456492,18,40673380,G,A,chr18:40673380,39673380,41673380
86,rs8087969,18,48683589,G,T,chr18:48683589,47683589,49683589
87,rs55818311,19,2341047,C,T,chr19:2341047,1341047,3341047
88,rs77351827,20,6006041,T,C,chr20:6006041,5006041,7006041


Run each sample and combine the result at the end

In [51]:
#csf_cardio
coord_path = f'{datadir}/PDBP_PPMI_proteomics_csf_cardio_coordinate.txt'
coords = pd.read_csv(coord_path, sep='\t')

#grab gene ID from coords file if position is within start and end in coords ---> coords['start'] <= row['pos'] <= coords['end']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & ((coords['start'] <= row['pos']) & (coords['end'] >= row['pos']))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_1 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['end']) >= row['pos_more']) | ((coords['start'] <= row['pos_less']) & (coords['end'] >= row['pos_less'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_2 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['start'] >= row['pos_less'])) | ((coords['end'] >= row['pos_less']) & (coords['end'] <= row['pos_more'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_3 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

print('Null ID in result_1: ',result_1.ID.isnull().sum())

#get non null values
print('Non Null ID in result_1: ',result_1[['ID']].T.stack().tolist())

print('Null ID in result_2: ',result_2.ID.isnull().sum())

#get non null values
print('Non Null ID in result_2: ',result_2[['ID']].T.stack().tolist())

print('Null ID in result_3: ',result_3.ID.isnull().sum())

#get non null values
print('Non Null ID in result_3: ',result_3[['ID']].T.stack().tolist())

Null ID in result_1:  85
Non Null ID in result_1:  ['PAM', 'GPNMB', 'CTSB', 'CRHR1', 'CRHR1']
Null ID in result_2:  89
Non Null ID in result_2:  ['CRHR1']
Null ID in result_3:  53
Non Null ID in result_3:  ['IL6R', 'IL6R', 'IL6R', 'FCGR2A', 'PM20D1', 'PM20D1', 'IL1RL1', 'SPON2', 'SPON2', 'QDPR', 'ART3', 'ART3', 'ART3', 'PAM', 'BAG6', 'IL6', 'GUSB', 'CTSB', 'TNFRSF10C', 'CCL27', 'GRK5', 'GRK5', 'PRCP', 'CNTN1', 'CNTN1', 'DIABLO', 'LGALS3', 'ENTPD5', 'CORO1A', 'AOC3', 'SOST', 'SOST', 'CRHR1', 'CRHR1', 'CRHR1', 'CANT1', 'THOP1']


In [53]:
#grab non duplicated values from 3 results

one = result_1[['ID']].T.stack().tolist()
two = result_2[['ID']].T.stack().tolist()
three = result_3[['ID']].T.stack().tolist()

three.extend(x for x in one if x not in three)
three.extend(x for x in two if x not in three)

csf_cardio_result = []
for i in three:
       if i not in csf_cardio_result:
            csf_cardio_result.append(i)
csf_cardio_result

['IL6R',
 'FCGR2A',
 'PM20D1',
 'IL1RL1',
 'SPON2',
 'QDPR',
 'ART3',
 'PAM',
 'BAG6',
 'IL6',
 'GUSB',
 'CTSB',
 'TNFRSF10C',
 'CCL27',
 'GRK5',
 'PRCP',
 'CNTN1',
 'DIABLO',
 'LGALS3',
 'ENTPD5',
 'CORO1A',
 'AOC3',
 'SOST',
 'CRHR1',
 'CANT1',
 'THOP1',
 'GPNMB']

In [54]:
#csf_inf
coord_path = f'{datadir}/PDBP_PPMI_proteomics_csf_inf_coordinate.txt'
coords = pd.read_csv(coord_path, sep='\t')

#grab gene ID from coords file if position is within start and end in coords ---> coords['start'] <= row['pos'] <= coords['end']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & ((coords['start'] <= row['pos']) & (coords['end'] >= row['pos']))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_1 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['end']) >= row['pos_more']) | ((coords['start'] <= row['pos_less']) & (coords['end'] >= row['pos_less'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_2 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['start'] >= row['pos_less'])) | ((coords['end'] >= row['pos_less']) & (coords['end'] <= row['pos_more'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_3 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

print('Null ID in result_1: ',result_1.ID.isnull().sum())

#get non null values
print('Non Null ID in result_1: ',result_1[['ID']].T.stack().tolist())

print('Null ID in result_2: ',result_2.ID.isnull().sum())

#get non null values
print('Non Null ID in result_2: ',result_2[['ID']].T.stack().tolist())

print('Null ID in result_3: ',result_3.ID.isnull().sum())

#get non null values
print('Non Null ID in result_3: ',result_3[['ID']].T.stack().tolist())

Null ID in result_1:  89
Non Null ID in result_1:  ['TBC1D5']
Null ID in result_2:  89
Non Null ID in result_2:  ['TBC1D5']
Null ID in result_3:  56
Non Null ID in result_3:  ['PKLR', 'PKLR', 'PKLR', 'CD84', 'FASLG', 'NFASC', 'NFASC', 'PARP1', 'IL1R2', 'TBC1D5', 'DAG1', 'HCLS1', 'LAMP3', 'LAP3', 'CXCL9', 'CXCL9', 'CXCL9', 'CXCL14', 'HLA-E', 'MPIG6B', 'LAMA4', 'CCN2', 'IL6', 'B4GALT1', 'PRDX3', 'PRDX3', 'PGF', 'PRSS8', 'EIF5A', 'HEXIM1', 'HEXIM1', 'HEXIM1', 'HEXIM1', 'NCLN']


In [55]:
#grab non duplicated values from 3 results

one = result_1[['ID']].T.stack().tolist()
two = result_2[['ID']].T.stack().tolist()
three = result_3[['ID']].T.stack().tolist()

three.extend(x for x in one if x not in three)
three.extend(x for x in two if x not in three)

csf_inf_result = []
for i in three:
       if i not in csf_inf_result:
            csf_inf_result.append(i)
csf_inf_result

['PKLR',
 'CD84',
 'FASLG',
 'NFASC',
 'PARP1',
 'IL1R2',
 'TBC1D5',
 'DAG1',
 'HCLS1',
 'LAMP3',
 'LAP3',
 'CXCL9',
 'CXCL14',
 'HLA-E',
 'MPIG6B',
 'LAMA4',
 'CCN2',
 'IL6',
 'B4GALT1',
 'PRDX3',
 'PGF',
 'PRSS8',
 'EIF5A',
 'HEXIM1',
 'NCLN']

In [56]:
#csf_neuro
coord_path = f'{datadir}/PDBP_PPMI_proteomics_csf_neuro_coordinate.txt'
coords = pd.read_csv(coord_path, sep='\t')

#grab gene ID from coords file if position is within start and end in coords ---> coords['start'] <= row['pos'] <= coords['end']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & ((coords['start'] <= row['pos']) & (coords['end'] >= row['pos']))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_1 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['end']) >= row['pos_more']) | ((coords['start'] <= row['pos_less']) & (coords['end'] >= row['pos_less'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_2 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['start'] >= row['pos_less'])) | ((coords['end'] >= row['pos_less']) & (coords['end'] <= row['pos_more'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_3 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

print('Null ID in result_1: ',result_1.ID.isnull().sum())

#get non null values
print('Non Null ID in result_1: ',result_1[['ID']].T.stack().tolist())

print('Null ID in result_2: ',result_2.ID.isnull().sum())

#get non null values
print('Non Null ID in result_2: ',result_2[['ID']].T.stack().tolist())

print('Null ID in result_3: ',result_3.ID.isnull().sum())

#get non null values
print('Non Null ID in result_3: ',result_3[['ID']].T.stack().tolist())

Null ID in result_1:  86
Non Null ID in result_1:  ['PMVK', 'BST1', 'SCARB2', 'BAG3']
Null ID in result_2:  90
Non Null ID in result_2:  []
Null ID in result_3:  61
Non Null ID in result_3:  ['PMVK', 'PMVK', 'PMVK', 'F11R', 'MYOC', 'IL1R1', 'BST1', 'NAAA', 'NAAA', 'NAAA', 'DDR1', 'TNXB', 'IL6', 'MSR1', 'SLC39A14', 'CCL19', 'BAG3', 'BAG3', 'CLEC14A', 'BMP4', 'SULT1A1', 'ITGAM', 'CLEC10A', 'DUSP3', 'DUSP3', 'FMNL1', 'FMNL1', 'MAPT', 'CHGB']


In [57]:
#grab non duplicated values from 3 results

one = result_1[['ID']].T.stack().tolist()
two = result_2[['ID']].T.stack().tolist()
three = result_3[['ID']].T.stack().tolist()

three.extend(x for x in one if x not in three)
three.extend(x for x in two if x not in three)

csf_neuro_result = []
for i in three:
       if i not in csf_neuro_result:
            csf_neuro_result.append(i)
csf_neuro_result

['PMVK',
 'F11R',
 'MYOC',
 'IL1R1',
 'BST1',
 'NAAA',
 'DDR1',
 'TNXB',
 'IL6',
 'MSR1',
 'SLC39A14',
 'CCL19',
 'BAG3',
 'CLEC14A',
 'BMP4',
 'SULT1A1',
 'ITGAM',
 'CLEC10A',
 'DUSP3',
 'FMNL1',
 'MAPT',
 'CHGB',
 'SCARB2']

In [58]:
#csf_onc
coord_path = f'{datadir}/PDBP_PPMI_proteomics_csf_onc_coordinate.txt'
coords = pd.read_csv(coord_path, sep='\t')

#grab gene ID from coords file if position is within start and end in coords ---> coords['start'] <= row['pos'] <= coords['end']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & ((coords['start'] <= row['pos']) & (coords['end'] >= row['pos']))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_1 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['end']) >= row['pos_more']) | ((coords['start'] <= row['pos_less']) & (coords['end'] >= row['pos_less'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_2 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['start'] >= row['pos_less'])) | ((coords['end'] >= row['pos_less']) & (coords['end'] <= row['pos_more'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_3 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

print('Null ID in result_1: ',result_1.ID.isnull().sum())

#get non null values
print('Non Null ID in result_1: ',result_1[['ID']].T.stack().tolist())

print('Null ID in result_2: ',result_2.ID.isnull().sum())

#get non null values
print('Non Null ID in result_2: ',result_2[['ID']].T.stack().tolist())

print('Null ID in result_3: ',result_3.ID.isnull().sum())

#get non null values
print('Non Null ID in result_3: ',result_3[['ID']].T.stack().tolist())

Null ID in result_1:  90
Non Null ID in result_1:  []
Null ID in result_2:  88
Non Null ID in result_2:  ['SLAMF6', 'SEPTIN9']
Null ID in result_3:  61
Non Null ID in result_3:  ['SCAMP3', 'SCAMP3', 'SCAMP3', 'SLAMF6', 'CNTN2', 'CNTN2', 'LEFTY2', 'IDUA', 'IDUA', 'CD38', 'TXNDC15', 'MOG', 'AIF1', 'VNN2', 'IL6', 'VPS37A', 'GFRA2', 'CDNF', 'ADM', 'PSMD9', 'SEZ6L2', 'STX4', 'STAT5B', 'PPY', 'PPY', 'GFAP', 'GFAP', 'CDC27', 'SEPTIN9']


In [59]:
#grab non duplicated values from 3 results

one = result_1[['ID']].T.stack().tolist()
two = result_2[['ID']].T.stack().tolist()
three = result_3[['ID']].T.stack().tolist()

three.extend(x for x in one if x not in three)
three.extend(x for x in two if x not in three)

csf_onc_result = []
for i in three:
       if i not in csf_onc_result:
            csf_onc_result.append(i)
csf_onc_result

['SCAMP3',
 'SLAMF6',
 'CNTN2',
 'LEFTY2',
 'IDUA',
 'CD38',
 'TXNDC15',
 'MOG',
 'AIF1',
 'VNN2',
 'IL6',
 'VPS37A',
 'GFRA2',
 'CDNF',
 'ADM',
 'PSMD9',
 'SEZ6L2',
 'STX4',
 'STAT5B',
 'PPY',
 'GFAP',
 'CDC27',
 'SEPTIN9']

In [60]:
#plasma_cardio
coord_path = f'{datadir}/PDBP_PPMI_proteomics_plasma_cardio_coordinate.txt'
coords = pd.read_csv(coord_path, sep='\t')

#grab gene ID from coords file if position is within start and end in coords ---> coords['start'] <= row['pos'] <= coords['end']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & ((coords['start'] <= row['pos']) & (coords['end'] >= row['pos']))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_1 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['end']) >= row['pos_more']) | ((coords['start'] <= row['pos_less']) & (coords['end'] >= row['pos_less'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_2 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['start'] >= row['pos_less'])) | ((coords['end'] >= row['pos_less']) & (coords['end'] <= row['pos_more'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_3 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

print('Null ID in result_1: ',result_1.ID.isnull().sum())

#get non null values
print('Non Null ID in result_1: ',result_1[['ID']].T.stack().tolist())

print('Null ID in result_2: ',result_2.ID.isnull().sum())

#get non null values
print('Non Null ID in result_2: ',result_2[['ID']].T.stack().tolist())

print('Null ID in result_3: ',result_3.ID.isnull().sum())

#get non null values
print('Non Null ID in result_3: ',result_3[['ID']].T.stack().tolist())

Null ID in result_1:  85
Non Null ID in result_1:  ['PAM', 'GPNMB', 'CTSB', 'CRHR1', 'CRHR1']
Null ID in result_2:  89
Non Null ID in result_2:  ['CRHR1']
Null ID in result_3:  53
Non Null ID in result_3:  ['IL6R', 'IL6R', 'IL6R', 'FCGR2A', 'PM20D1', 'PM20D1', 'IL1RL1', 'SPON2', 'SPON2', 'QDPR', 'ART3', 'ART3', 'ART3', 'PAM', 'BAG6', 'IL6', 'GUSB', 'CTSB', 'DOK2', 'CCL27', 'GRK5', 'GRK5', 'PRCP', 'CNTN1', 'CNTN1', 'DIABLO', 'LGALS3', 'ENTPD5', 'CORO1A', 'AOC3', 'SOST', 'SOST', 'CRHR1', 'CRHR1', 'CRHR1', 'CANT1', 'THOP1']


In [61]:
#grab non duplicated values from 3 results

one = result_1[['ID']].T.stack().tolist()
two = result_2[['ID']].T.stack().tolist()
three = result_3[['ID']].T.stack().tolist()

three.extend(x for x in one if x not in three)
three.extend(x for x in two if x not in three)

plasma_cardio_result = []
for i in three:
       if i not in plasma_cardio_result:
            plasma_cardio_result.append(i)
plasma_cardio_result

['IL6R',
 'FCGR2A',
 'PM20D1',
 'IL1RL1',
 'SPON2',
 'QDPR',
 'ART3',
 'PAM',
 'BAG6',
 'IL6',
 'GUSB',
 'CTSB',
 'DOK2',
 'CCL27',
 'GRK5',
 'PRCP',
 'CNTN1',
 'DIABLO',
 'LGALS3',
 'ENTPD5',
 'CORO1A',
 'AOC3',
 'SOST',
 'CRHR1',
 'CANT1',
 'THOP1',
 'GPNMB']

In [62]:
#plasma_inf
coord_path = f'{datadir}/PDBP_PPMI_proteomics_plasma_inf_coordinate.txt'
coords = pd.read_csv(coord_path, sep='\t')

#grab gene ID from coords file if position is within start and end in coords ---> coords['start'] <= row['pos'] <= coords['end']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & ((coords['start'] <= row['pos']) & (coords['end'] >= row['pos']))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_1 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['end']) >= row['pos_more']) | ((coords['start'] <= row['pos_less']) & (coords['end'] >= row['pos_less'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_2 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['start'] >= row['pos_less'])) | ((coords['end'] >= row['pos_less']) & (coords['end'] <= row['pos_more'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_3 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

print('Null ID in result_1: ',result_1.ID.isnull().sum())

#get non null values
print('Non Null ID in result_1: ',result_1[['ID']].T.stack().tolist())

print('Null ID in result_2: ',result_2.ID.isnull().sum())

#get non null values
print('Non Null ID in result_2: ',result_2[['ID']].T.stack().tolist())

print('Null ID in result_3: ',result_3.ID.isnull().sum())

#get non null values
print('Non Null ID in result_3: ',result_3[['ID']].T.stack().tolist())

Null ID in result_1:  89
Non Null ID in result_1:  ['TBC1D5']
Null ID in result_2:  89
Non Null ID in result_2:  ['TBC1D5']
Null ID in result_3:  55
Non Null ID in result_3:  ['PKLR', 'PKLR', 'PKLR', 'CD84', 'FASLG', 'NFASC', 'NFASC', 'PARP1', 'IL1R2', 'TBC1D5', 'DAG1', 'HCLS1', 'LAMP3', 'LAP3', 'CXCL9', 'CXCL9', 'CXCL9', 'CXCL14', 'HLA-E', 'MPIG6B', 'LAMA4', 'CCN2', 'IL6', 'B4GALT1', 'PRDX3', 'PRDX3', 'PGF', 'LAT', 'PRSS8', 'EIF5A', 'HEXIM1', 'HEXIM1', 'HEXIM1', 'HEXIM1', 'NCLN']


In [63]:
#grab non duplicated values from 3 results

one = result_1[['ID']].T.stack().tolist()
two = result_2[['ID']].T.stack().tolist()
three = result_3[['ID']].T.stack().tolist()

three.extend(x for x in one if x not in three)
three.extend(x for x in two if x not in three)

plasma_inf_result = []
for i in three:
       if i not in plasma_inf_result:
            plasma_inf_result.append(i)
plasma_inf_result

['PKLR',
 'CD84',
 'FASLG',
 'NFASC',
 'PARP1',
 'IL1R2',
 'TBC1D5',
 'DAG1',
 'HCLS1',
 'LAMP3',
 'LAP3',
 'CXCL9',
 'CXCL14',
 'HLA-E',
 'MPIG6B',
 'LAMA4',
 'CCN2',
 'IL6',
 'B4GALT1',
 'PRDX3',
 'PGF',
 'LAT',
 'PRSS8',
 'EIF5A',
 'HEXIM1',
 'NCLN']

In [64]:
#plasma_neuro
coord_path = f'{datadir}/PDBP_PPMI_proteomics_plasma_neuro_coordinate.txt'
coords = pd.read_csv(coord_path, sep='\t')

#grab gene ID from coords file if position is within start and end in coords ---> coords['start'] <= row['pos'] <= coords['end']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & ((coords['start'] <= row['pos']) & (coords['end'] >= row['pos']))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_1 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['end']) >= row['pos_more']) | ((coords['start'] <= row['pos_less']) & (coords['end'] >= row['pos_less'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_2 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['start'] >= row['pos_less'])) | ((coords['end'] >= row['pos_less']) & (coords['end'] <= row['pos_more'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_3 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

print('Null ID in result_1: ',result_1.ID.isnull().sum())

#get non null values
print('Non Null ID in result_1: ',result_1[['ID']].T.stack().tolist())

print('Null ID in result_2: ',result_2.ID.isnull().sum())

#get non null values
print('Non Null ID in result_2: ',result_2[['ID']].T.stack().tolist())

print('Null ID in result_3: ',result_3.ID.isnull().sum())

#get non null values
print('Non Null ID in result_3: ',result_3[['ID']].T.stack().tolist())

Null ID in result_1:  86
Non Null ID in result_1:  ['PMVK', 'BST1', 'SCARB2', 'BAG3']
Null ID in result_2:  89
Non Null ID in result_2:  ['KRT14']
Null ID in result_3:  61
Non Null ID in result_3:  ['PMVK', 'PMVK', 'PMVK', 'F11R', 'MYOC', 'IL1R1', 'BST1', 'NAAA', 'NAAA', 'NAAA', 'DDR1', 'TNXB', 'IL6', 'MSR1', 'SLC39A14', 'CCL19', 'BAG3', 'BAG3', 'CLEC14A', 'BMP4', 'SULT1A1', 'ITGAM', 'CLEC10A', 'KRT14', 'DUSP3', 'DUSP3', 'FMNL1', 'FMNL1', 'CHGB']


In [65]:
#grab non duplicated values from 3 results

one = result_1[['ID']].T.stack().tolist()
two = result_2[['ID']].T.stack().tolist()
three = result_3[['ID']].T.stack().tolist()

three.extend(x for x in one if x not in three)
three.extend(x for x in two if x not in three)

plasma_neuro_result = []
for i in three:
       if i not in plasma_neuro_result:
            plasma_neuro_result.append(i)
plasma_neuro_result

['PMVK',
 'F11R',
 'MYOC',
 'IL1R1',
 'BST1',
 'NAAA',
 'DDR1',
 'TNXB',
 'IL6',
 'MSR1',
 'SLC39A14',
 'CCL19',
 'BAG3',
 'CLEC14A',
 'BMP4',
 'SULT1A1',
 'ITGAM',
 'CLEC10A',
 'KRT14',
 'DUSP3',
 'FMNL1',
 'CHGB',
 'SCARB2']

In [66]:
#plasma_onc
coord_path = f'{datadir}/PDBP_PPMI_proteomics_plasma_onc_coordinate.txt'
coords = pd.read_csv(coord_path, sep='\t')

#grab gene ID from coords file if position is within start and end in coords ---> coords['start'] <= row['pos'] <= coords['end']

from numpy import nan

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & ((coords['start'] <= row['pos']) & (coords['end'] >= row['pos']))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_1 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['end']) >= row['pos_more']) | ((coords['start'] <= row['pos_less']) & (coords['end'] >= row['pos_less'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_2 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

def match_labels(row):
    curr_df = coords[(coords['X.Chr'] == row['chr']) & (((coords['start'] <= row['pos_more']) & (coords['start'] >= row['pos_less'])) | ((coords['end'] >= row['pos_less']) & (coords['end'] <= row['pos_more'])))]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.NaN

    return row

result_3 = pd_hits_pos_2.apply(lambda x:match_labels(x),axis=1)

print('Null ID in result_1: ',result_1.ID.isnull().sum())

#get non null values
print('Non Null ID in result_1: ',result_1[['ID']].T.stack().tolist())

print('Null ID in result_2: ',result_2.ID.isnull().sum())

#get non null values
print('Non Null ID in result_2: ',result_2[['ID']].T.stack().tolist())

print('Null ID in result_3: ',result_3.ID.isnull().sum())

#get non null values
print('Non Null ID in result_3: ',result_3[['ID']].T.stack().tolist())

Null ID in result_1:  90
Non Null ID in result_1:  []
Null ID in result_2:  88
Non Null ID in result_2:  ['SLAMF6', 'SEPTIN9']
Null ID in result_3:  57
Non Null ID in result_3:  ['SCAMP3', 'SCAMP3', 'SCAMP3', 'SLAMF6', 'CNTN2', 'CNTN2', 'LEFTY2', 'IDUA', 'IDUA', 'CD38', 'USO1', 'USO1', 'USO1', 'TXNDC15', 'MOG', 'AIF1', 'VNN2', 'IL6', 'VPS37A', 'GFRA2', 'NUDT2', 'CDNF', 'ADM', 'PSMD9', 'SEZ6L2', 'STX4', 'STAT5B', 'PPY', 'PPY', 'GFAP', 'GFAP', 'CDC27', 'SEPTIN9']


In [67]:
#grab non duplicated values from 3 results

one = result_1[['ID']].T.stack().tolist()
two = result_2[['ID']].T.stack().tolist()
three = result_3[['ID']].T.stack().tolist()

three.extend(x for x in one if x not in three)
three.extend(x for x in two if x not in three)

plasma_onc_result = []
for i in three:
       if i not in plasma_onc_result:
            plasma_onc_result.append(i)
plasma_onc_result

['SCAMP3',
 'SLAMF6',
 'CNTN2',
 'LEFTY2',
 'IDUA',
 'CD38',
 'USO1',
 'TXNDC15',
 'MOG',
 'AIF1',
 'VNN2',
 'IL6',
 'VPS37A',
 'GFRA2',
 'NUDT2',
 'CDNF',
 'ADM',
 'PSMD9',
 'SEZ6L2',
 'STX4',
 'STAT5B',
 'PPY',
 'GFAP',
 'CDC27',
 'SEPTIN9']

In [70]:
#read proteomics result 

#read gwas significant

#put the result into a new dataframe 
csf_cardio = pd.DataFrame({'csf_cardio':csf_cardio_result})
csf_inf = pd.DataFrame({'csf_inf':csf_inf_result})
csf_neuro = pd.DataFrame({'csf_neuro':csf_neuro_result})
csf_onc = pd.DataFrame({'csf_onc':csf_onc_result})
plasma_cardio = pd.DataFrame({'plasma_cardio':plasma_cardio_result})
plasma_inf = pd.DataFrame({'plasma_inf':plasma_inf_result})
plasma_neuro = pd.DataFrame({'plasma_neuro':plasma_neuro_result})
plasma_onc = pd.DataFrame({'plasma_onc':plasma_onc_result})

proteomics_result = pd.concat([csf_cardio, csf_inf, csf_neuro, csf_onc, plasma_cardio, plasma_inf, plasma_neuro, plasma_onc], axis=1) 
proteomics_result 
#these are genes +/- 1MB from the 90 parkinson's GWAS hits

,csf_cardio,csf_inf,csf_neuro,csf_onc,plasma_cardio,plasma_inf,plasma_neuro,plasma_onc
0,IL6R,PKLR,PMVK,SCAMP3,IL6R,PKLR,PMVK,SCAMP3
1,FCGR2A,CD84,F11R,SLAMF6,FCGR2A,CD84,F11R,SLAMF6
2,PM20D1,FASLG,MYOC,CNTN2,PM20D1,FASLG,MYOC,CNTN2
3,IL1RL1,NFASC,IL1R1,LEFTY2,IL1RL1,NFASC,IL1R1,LEFTY2
4,SPON2,PARP1,BST1,IDUA,SPON2,PARP1,BST1,IDUA
5,QDPR,IL1R2,NAAA,CD38,QDPR,IL1R2,NAAA,CD38
6,ART3,TBC1D5,DDR1,TXNDC15,ART3,TBC1D5,DDR1,USO1
7,PAM,DAG1,TNXB,MOG,PAM,DAG1,TNXB,TXNDC15
8,BAG6,HCLS1,IL6,AIF1,BAG6,HCLS1,IL6,MOG
9,IL6,LAMP3,MSR1,VNN2,IL6,LAMP3,MSR1,AIF1


In [71]:
proteomics_result.to_csv(r'./data_folder/proteomics_pd_hits.csv',index=False)